In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from tensorboard_logger import configure, log_value
import os


logdir = 'C:\\Users\\kkb6\\Desktop\\PlayListCurator\\model-training\\logs\\'
checkpointdir = 'C:\\Users\\kkb6\\Desktop\\PlayListCurator\\model-training\\checkpoints\\run-1\\'


In [2]:
logdir

'C:\\Users\\kkb6\\Desktop\\PlayListCurator\\model-training\\logs\\'

In [3]:
checkpointdir

'C:\\Users\\kkb6\\Desktop\\PlayListCurator\\model-training\\checkpoints\\run-1\\'

In [4]:
y = np.load("label_array.npy")
x = np.load("data_array.npy")
np.random.seed(5)
index = np.random.permutation(x.shape[0])
x = x[index]
y = y[index]


In [5]:
#List of tuples
X = []
for feature, label in zip(x,y):
    X.append((feature,label))

In [6]:
def data_iterator(fs, seed=0, batch_size=4, num_fs=0, num_batch=0):

    NORMALIZE = True
    
    fs_a = fs[:num_fs]
    fs = fs[num_fs:]
    num = batch_size - num_batch
    num_itr = int(len(fs)/num)
    num_itr_a = int(len(fs_a)/num_batch) if num_batch != 0 else 1
    while True:
        idxs = list(range(0, len(fs)))
        idxs_a = list(range(0, len(fs_a)))
        np.random.shuffle(idxs)
        np.random.shuffle(idxs_a)

        for batch_idx in range(0, num_itr):
            current_indices = idxs[batch_idx*num:batch_idx*num + num]
            batch_idx = batch_idx % num_itr_a
            current_indices_a = idxs_a[batch_idx*num_batch:batch_idx*num_batch + num_batch]

            features_batch = []
            labels_batch = []
            
            for j in range(batch_size):
                if j < num:
                    i = current_indices[j]
                    x, label = fs[i]
                else:
                    i = current_indices_a[j - num]
                    x, label = fs_a[i]
                                      
                #Normalization - Subtrating by the mean and dividing by the standard deviation
                if NORMALIZE:

                    x = (x - x.mean())/ x.std()

                    x = x/ (np.linalg.norm(x))
                    
                
                features_batch.append(x)
                labels_batch.append(label)

            features_batch = np.array(features_batch)
            labels_batch = np.array(labels_batch)

            yield features_batch, labels_batch


In [7]:
class PlayListCurator(nn.Module):

    def __init__(self):
        super(PlayListCurator, self).__init__()
        self.fc1 = nn.Linear(445, 100)
        #self.fc2 = nn.Linear(100, 84)
        self.fc3 = nn.Linear(100, 14)


    def forward(self, x):
        # If the size is a square you can only specify a single number
        x = F.sigmoid(self.fc1(x)) #hidden1
        #x = F.relu(self.fc2(x)) #hidden2
        x = F.log_softmax(self.fc3(x)) #output
        return x
    

In [8]:
date, tag = '20190418', 'run-PlayList-Curator'

comments = """First training"""

In [9]:
batch_size = 4

length = len(X)


train_split = 0.80
valid_split = 0.10
test_split = 0.10


indices1 = round(train_split * length)
indices2 = indices1 + round(valid_split*length)

train_data = X[:indices1]

valid_data = X[indices1:indices2]

test_data = X[indices2:]

In [10]:
a_train = data_iterator(train_data,batch_size=batch_size)

a_valid = data_iterator(valid_data,batch_size=batch_size)

a_test = data_iterator(test_data,batch_size=batch_size)

In [11]:
a_train.__next__()

(array([[-0.00224745, -0.00224853,  0.99887562, ..., -0.00225143,
         -0.00225143, -0.00225143],
        [-0.00224423, -0.0022429 ,  0.99887532, ..., -0.00225177,
         -0.00225177, -0.00225177],
        [-0.00224605, -0.00224579,  0.99887499, ..., -0.00225312,
         -0.00225312, -0.00225312],
        [-0.00225221, -0.00224832,  0.99887536, ..., -0.00225237,
         -0.00225237, -0.00225237]]), array([7., 9., 9., 5.]))

In [12]:
net = PlayListCurator()
net = net.double()

In [13]:
_ = net

optimizer = optim.Adam(net.parameters(), weight_decay=8e-6, lr=0.0001)

criterion = nn.CrossEntropyLoss()
i = 0
best_valid_loss = 1000000

In [14]:
configure(logdir + '\\run-1')


In [15]:
valid_iters = 100
for _ in range(10000):
    i += 1
    log = i%3 == 0
    is_valid = i%100 == 0 or i == 20
    
    x, labs = a_train.__next__()
    x = Variable(torch.from_numpy(x))
    labs = Variable(torch.from_numpy(labs))
    labs = labs.long()
    
    output  = net(x)

    optimizer.zero_grad()

    loss = criterion(output, labs)
    log_value('training_loss',loss,i)
    
    print("Iteration: {}, Loss:{:.2f} ".format(i + 1,loss)) 
    
    loss.backward()
 
    optimizer.step()
        
    if is_valid:
        losses = []
        for __ in range(valid_iters):
            x, labs = a_valid.__next__()
            x = Variable(torch.from_numpy(x))
            labs= Variable(torch.from_numpy(labs))
            labs = labs.long()
            
            output  = net(x)
            v_loss = criterion(output, labs).detach()
    
            losses.append(v_loss)

        valid_loss = np.mean(losses)
        log_value('valid_loss', valid_loss, i)
              
        if valid_loss < best_valid_loss:
            torch.save(net.state_dict(), os.path.join(checkpointdir, tag + '_weights_%d.ckpt'%i))
            print(os.path.join(checkpointdir, tag + '_weights_%d.ckpt'%i))
            best_valid_loss = valid_loss
       
    if i % 250 == 0:
        torch.save(net.state_dict(), os.path.join(checkpointdir, tag + '_weights_%d.ckpt'%i))
        print(os.path.join(checkpointdir, tag + '_weights_%d.ckpt'%i))

C:\Users\kkb6\AppData\Local\Continuum\anaconda3\envs\texture\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\kkb6\AppData\Local\Continuum\anaconda3\envs\texture\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


Iteration: 2, Loss:2.77 
Iteration: 3, Loss:2.52 
Iteration: 4, Loss:2.73 
Iteration: 5, Loss:2.68 
Iteration: 6, Loss:2.54 
Iteration: 7, Loss:2.63 
Iteration: 8, Loss:2.73 
Iteration: 9, Loss:2.91 
Iteration: 10, Loss:2.81 
Iteration: 11, Loss:2.98 
Iteration: 12, Loss:2.57 
Iteration: 13, Loss:2.63 
Iteration: 14, Loss:2.72 
Iteration: 15, Loss:2.72 
Iteration: 16, Loss:2.31 
Iteration: 17, Loss:2.80 
Iteration: 18, Loss:2.94 
Iteration: 19, Loss:2.57 
Iteration: 20, Loss:2.80 
Iteration: 21, Loss:2.83 
C:\Users\kkb6\Desktop\PlayListCurator\model-training\checkpoints\run-1\run-PlayList-Curator_weights_20.ckpt
Iteration: 22, Loss:2.78 
Iteration: 23, Loss:2.68 
Iteration: 24, Loss:2.78 
Iteration: 25, Loss:2.93 
Iteration: 26, Loss:2.70 
Iteration: 27, Loss:2.50 
Iteration: 28, Loss:2.38 
Iteration: 29, Loss:2.74 
Iteration: 30, Loss:2.79 
Iteration: 31, Loss:2.61 
Iteration: 32, Loss:2.67 
Iteration: 33, Loss:3.00 
Iteration: 34, Loss:2.52 
Iteration: 35, Loss:2.54 
Iteration: 36, L

Iteration: 293, Loss:2.46 
Iteration: 294, Loss:2.52 
Iteration: 295, Loss:2.71 
Iteration: 296, Loss:2.62 
Iteration: 297, Loss:2.57 
Iteration: 298, Loss:2.74 
Iteration: 299, Loss:2.42 
Iteration: 300, Loss:2.52 
Iteration: 301, Loss:2.77 
C:\Users\kkb6\Desktop\PlayListCurator\model-training\checkpoints\run-1\run-PlayList-Curator_weights_300.ckpt
Iteration: 302, Loss:2.63 
Iteration: 303, Loss:2.63 
Iteration: 304, Loss:2.80 
Iteration: 305, Loss:2.64 
Iteration: 306, Loss:2.68 
Iteration: 307, Loss:2.57 
Iteration: 308, Loss:2.68 
Iteration: 309, Loss:2.39 
Iteration: 310, Loss:2.30 
Iteration: 311, Loss:2.19 
Iteration: 312, Loss:2.64 
Iteration: 313, Loss:2.67 
Iteration: 314, Loss:2.64 
Iteration: 315, Loss:2.76 
Iteration: 316, Loss:2.22 
Iteration: 317, Loss:2.56 
Iteration: 318, Loss:2.68 
Iteration: 319, Loss:2.47 
Iteration: 320, Loss:2.24 
Iteration: 321, Loss:2.59 
Iteration: 322, Loss:2.52 
Iteration: 323, Loss:2.51 
Iteration: 324, Loss:2.67 
Iteration: 325, Loss:2.33 


Iteration: 580, Loss:2.67 
Iteration: 581, Loss:2.69 
Iteration: 582, Loss:2.70 
Iteration: 583, Loss:2.37 
Iteration: 584, Loss:2.73 
Iteration: 585, Loss:2.50 
Iteration: 586, Loss:2.30 
Iteration: 587, Loss:2.77 
Iteration: 588, Loss:2.71 
Iteration: 589, Loss:2.50 
Iteration: 590, Loss:2.35 
Iteration: 591, Loss:2.67 
Iteration: 592, Loss:2.54 
Iteration: 593, Loss:2.45 
Iteration: 594, Loss:2.59 
Iteration: 595, Loss:2.32 
Iteration: 596, Loss:2.21 
Iteration: 597, Loss:2.82 
Iteration: 598, Loss:2.60 
Iteration: 599, Loss:2.60 
Iteration: 600, Loss:2.48 
Iteration: 601, Loss:2.55 
Iteration: 602, Loss:2.65 
Iteration: 603, Loss:2.34 
Iteration: 604, Loss:2.63 
Iteration: 605, Loss:2.66 
Iteration: 606, Loss:2.77 
Iteration: 607, Loss:2.83 
Iteration: 608, Loss:2.64 
Iteration: 609, Loss:2.49 
Iteration: 610, Loss:2.47 
Iteration: 611, Loss:2.81 
Iteration: 612, Loss:2.61 
Iteration: 613, Loss:2.79 
Iteration: 614, Loss:2.47 
Iteration: 615, Loss:2.51 
Iteration: 616, Loss:2.48 
I

Iteration: 879, Loss:2.90 
Iteration: 880, Loss:2.65 
Iteration: 881, Loss:2.63 
Iteration: 882, Loss:2.74 
Iteration: 883, Loss:2.45 
Iteration: 884, Loss:2.77 
Iteration: 885, Loss:2.62 
Iteration: 886, Loss:2.77 
Iteration: 887, Loss:2.59 
Iteration: 888, Loss:2.41 
Iteration: 889, Loss:2.62 
Iteration: 890, Loss:2.60 
Iteration: 891, Loss:2.41 
Iteration: 892, Loss:2.70 
Iteration: 893, Loss:2.61 
Iteration: 894, Loss:2.52 
Iteration: 895, Loss:2.63 
Iteration: 896, Loss:2.28 
Iteration: 897, Loss:2.96 
Iteration: 898, Loss:2.29 
Iteration: 899, Loss:2.74 
Iteration: 900, Loss:2.72 
Iteration: 901, Loss:2.39 
Iteration: 902, Loss:2.40 
Iteration: 903, Loss:2.87 
Iteration: 904, Loss:2.37 
Iteration: 905, Loss:2.32 
Iteration: 906, Loss:2.40 
Iteration: 907, Loss:2.68 
Iteration: 908, Loss:2.60 
Iteration: 909, Loss:2.54 
Iteration: 910, Loss:2.56 
Iteration: 911, Loss:2.60 
Iteration: 912, Loss:2.22 
Iteration: 913, Loss:2.35 
Iteration: 914, Loss:2.86 
Iteration: 915, Loss:2.50 
I

...

Iteration: 8640, Loss:2.28 
Iteration: 8641, Loss:2.53 
Iteration: 8642, Loss:2.70 
Iteration: 8643, Loss:2.80 
Iteration: 8644, Loss:2.88 
Iteration: 8645, Loss:2.95 
Iteration: 8646, Loss:2.67 
Iteration: 8647, Loss:2.33 
Iteration: 8648, Loss:2.98 
Iteration: 8649, Loss:2.88 
Iteration: 8650, Loss:2.84 
Iteration: 8651, Loss:3.07 
Iteration: 8652, Loss:2.52 
Iteration: 8653, Loss:2.74 
Iteration: 8654, Loss:2.39 
Iteration: 8655, Loss:2.41 
Iteration: 8656, Loss:2.73 
Iteration: 8657, Loss:2.84 
Iteration: 8658, Loss:2.49 
Iteration: 8659, Loss:2.57 
Iteration: 8660, Loss:2.83 
Iteration: 8661, Loss:2.62 
Iteration: 8662, Loss:2.68 
Iteration: 8663, Loss:2.83 
Iteration: 8664, Loss:2.32 
Iteration: 8665, Loss:2.63 
Iteration: 8666, Loss:2.22 
Iteration: 8667, Loss:2.39 
Iteration: 8668, Loss:2.49 
Iteration: 8669, Loss:2.75 
Iteration: 8670, Loss:2.84 
Iteration: 8671, Loss:2.10 
Iteration: 8672, Loss:2.58 
Iteration: 8673, Loss:2.47 
Iteration: 8674, Loss:2.42 
Iteration: 8675, Los

Iteration: 8928, Loss:2.52 
Iteration: 8929, Loss:2.56 
Iteration: 8930, Loss:3.01 
Iteration: 8931, Loss:2.55 
Iteration: 8932, Loss:2.84 
Iteration: 8933, Loss:2.78 
Iteration: 8934, Loss:2.47 
Iteration: 8935, Loss:2.39 
Iteration: 8936, Loss:2.51 
Iteration: 8937, Loss:2.82 
Iteration: 8938, Loss:2.67 
Iteration: 8939, Loss:2.25 
Iteration: 8940, Loss:2.46 
Iteration: 8941, Loss:2.46 
Iteration: 8942, Loss:2.76 
Iteration: 8943, Loss:2.77 
Iteration: 8944, Loss:2.74 
Iteration: 8945, Loss:2.43 
Iteration: 8946, Loss:2.65 
Iteration: 8947, Loss:2.08 
Iteration: 8948, Loss:2.71 
Iteration: 8949, Loss:2.24 
Iteration: 8950, Loss:2.66 
Iteration: 8951, Loss:2.39 
Iteration: 8952, Loss:2.33 
Iteration: 8953, Loss:2.30 
Iteration: 8954, Loss:2.29 
Iteration: 8955, Loss:2.52 
Iteration: 8956, Loss:2.70 
Iteration: 8957, Loss:2.60 
Iteration: 8958, Loss:2.45 
Iteration: 8959, Loss:2.30 
Iteration: 8960, Loss:2.45 
Iteration: 8961, Loss:2.63 
Iteration: 8962, Loss:2.66 
Iteration: 8963, Los

Iteration: 9216, Loss:2.85 
Iteration: 9217, Loss:2.70 
Iteration: 9218, Loss:2.74 
Iteration: 9219, Loss:2.45 
Iteration: 9220, Loss:2.16 
Iteration: 9221, Loss:2.83 
Iteration: 9222, Loss:2.04 
Iteration: 9223, Loss:2.66 
Iteration: 9224, Loss:2.80 
Iteration: 9225, Loss:2.51 
Iteration: 9226, Loss:2.61 
Iteration: 9227, Loss:2.68 
Iteration: 9228, Loss:2.59 
Iteration: 9229, Loss:2.79 
Iteration: 9230, Loss:2.49 
Iteration: 9231, Loss:2.14 
Iteration: 9232, Loss:2.18 
Iteration: 9233, Loss:2.79 
Iteration: 9234, Loss:2.65 
Iteration: 9235, Loss:2.75 
Iteration: 9236, Loss:2.31 
Iteration: 9237, Loss:2.02 
Iteration: 9238, Loss:2.38 
Iteration: 9239, Loss:2.24 
Iteration: 9240, Loss:2.41 
Iteration: 9241, Loss:2.58 
Iteration: 9242, Loss:2.78 
Iteration: 9243, Loss:2.95 
Iteration: 9244, Loss:2.98 
Iteration: 9245, Loss:2.51 
Iteration: 9246, Loss:2.47 
Iteration: 9247, Loss:2.74 
Iteration: 9248, Loss:2.37 
Iteration: 9249, Loss:2.45 
Iteration: 9250, Loss:2.80 
Iteration: 9251, Los

C:\Users\kkb6\Desktop\PlayListCurator\model-training\checkpoints\run-1\run-PlayList-Curator_weights_9500.ckpt
Iteration: 9502, Loss:2.35 
Iteration: 9503, Loss:2.26 
Iteration: 9504, Loss:2.40 
Iteration: 9505, Loss:2.46 
Iteration: 9506, Loss:2.61 
Iteration: 9507, Loss:2.62 
Iteration: 9508, Loss:2.70 
Iteration: 9509, Loss:2.94 
Iteration: 9510, Loss:2.61 
Iteration: 9511, Loss:2.74 
Iteration: 9512, Loss:2.62 
Iteration: 9513, Loss:2.66 
Iteration: 9514, Loss:2.28 
Iteration: 9515, Loss:2.46 
Iteration: 9516, Loss:2.62 
Iteration: 9517, Loss:2.71 
Iteration: 9518, Loss:2.16 
Iteration: 9519, Loss:2.56 
Iteration: 9520, Loss:2.31 
Iteration: 9521, Loss:2.41 
Iteration: 9522, Loss:2.76 
Iteration: 9523, Loss:2.77 
Iteration: 9524, Loss:2.38 
Iteration: 9525, Loss:2.66 
Iteration: 9526, Loss:2.77 
Iteration: 9527, Loss:2.85 
Iteration: 9528, Loss:2.61 
Iteration: 9529, Loss:2.90 
Iteration: 9530, Loss:2.73 
Iteration: 9531, Loss:2.33 
Iteration: 9532, Loss:2.40 
Iteration: 9533, Loss:

Iteration: 9786, Loss:2.66 
Iteration: 9787, Loss:2.34 
Iteration: 9788, Loss:2.86 
Iteration: 9789, Loss:2.67 
Iteration: 9790, Loss:2.13 
Iteration: 9791, Loss:2.27 
Iteration: 9792, Loss:2.30 
Iteration: 9793, Loss:2.68 
Iteration: 9794, Loss:3.00 
Iteration: 9795, Loss:2.68 
Iteration: 9796, Loss:2.52 
Iteration: 9797, Loss:2.28 
Iteration: 9798, Loss:2.47 
Iteration: 9799, Loss:2.59 
Iteration: 9800, Loss:2.39 
Iteration: 9801, Loss:2.63 
Iteration: 9802, Loss:2.57 
Iteration: 9803, Loss:2.77 
Iteration: 9804, Loss:2.61 
Iteration: 9805, Loss:2.61 
Iteration: 9806, Loss:2.68 
Iteration: 9807, Loss:2.77 
Iteration: 9808, Loss:2.23 
Iteration: 9809, Loss:2.60 
Iteration: 9810, Loss:2.48 
Iteration: 9811, Loss:2.79 
Iteration: 9812, Loss:2.40 
Iteration: 9813, Loss:2.51 
Iteration: 9814, Loss:2.86 
Iteration: 9815, Loss:2.35 
Iteration: 9816, Loss:2.48 
Iteration: 9817, Loss:2.46 
Iteration: 9818, Loss:2.30 
Iteration: 9819, Loss:2.55 
Iteration: 9820, Loss:2.71 
Iteration: 9821, Los